<a href="https://colab.research.google.com/github/wandb/examples/blob/master/colabs/torchtune/torchtune_and_wandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{torchtune-colab} -->

<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />
<!--- @wandbcode{torchtune-colab} -->

# Getting Started with torchtune and Weigths & Biases

In this notebook you will learn how to use [torchtune](https://github.com/pytorch/torchtune) with [Weights & Biases](https://wandb.ai) to monitor your training runs.

> You need to select a machine a GPU, go to Runtime > Change runtime type > select a GPU (L40, A100 ideally)

## Setup the libraries

In [ ]:
!git clone --depth 1 https://github.com/pytorch/torchtune

In [ ]:
cd torchtune/

In [ ]:
!python -m pip install -qqq ".[dev]"

In [ ]:
import wandb
wandb.login()

## Download a Model
We will download a model from the Hugging Face Hub.
> you will need to provide an access token or call `huggingface-cli login`

Download a model checkpoint using the provided `tune download` CLI



In [ ]:
!tune download mistralai/Mistral-7B-v0.1 \
    --output-dir /tmp/Mistral-7B-v0.1/ \
    --hf-token <HF_TOKEN>

Let's create a torchtune config that enables W&B, to do so, we can grab the original Mistral 7B LoRA recipe and change the following lines to use W&B as our `metric_logger`:
```yaml
# Logging
metric_logger:
  _component_: torchtune.utils.metric_logging.WandBLogger # <---You only need this to enable W&B
  project: mistral_lora # <--- The W&B project to save our logs to
log_every_n_steps: 1

```

Let's save a modified version of the recipe using `%%writefile`:

In [ ]:
%%writefile mistral_wandb_lora.yaml
tokenizer:
  _component_: torchtune.models.mistral.mistral_tokenizer
  path: /tmp/Mistral-7B-v0.1/tokenizer.model

# Dataset
dataset:
  _component_: torchtune.datasets.alpaca_dataset
  train_on_input: True
seed: null
shuffle: True

# Model Arguments
model:
  _component_: torchtune.models.mistral.lora_mistral_7b
  lora_attn_modules: ['q_proj', 'k_proj', 'v_proj']
  apply_lora_to_mlp: True
  apply_lora_to_output: True
  lora_rank: 64
  lora_alpha: 16

checkpointer:
  _component_: torchtune.utils.FullModelHFCheckpointer
  checkpoint_dir: /tmp/Mistral-7B-v0.1
  checkpoint_files: [
    pytorch_model-00001-of-00002.bin,
    pytorch_model-00002-of-00002.bin
  ]
  recipe_checkpoint: null
  output_dir: /tmp/Mistral-7B-v0.1
  model_type: MISTRAL
resume_from_checkpoint: False

optimizer:
  _component_: torch.optim.AdamW
  lr: 2e-5

lr_scheduler:
  _component_: torchtune.modules.get_cosine_schedule_with_warmup
  num_warmup_steps: 100

loss:
  _component_: torch.nn.CrossEntropyLoss

# Fine-tuning arguments
batch_size: 2
epochs: 1
max_steps_per_epoch: 100
gradient_accumulation_steps: 2
compile: False

# Training env
device: cuda

# Memory management
enable_activation_checkpointing: True

# Reduced precision
dtype: bf16
############################### Enable W&B #####################################
################################################################################
# Logging
metric_logger:
  _component_: torchtune.utils.metric_logging.WandBLogger # <---You only need this to enable W&B
  project: mistral_lora # <--- The W&B project to save our logs to
log_every_n_steps: 1
################################################################################
################################################################################
output_dir: /tmp/Mistral-7B-v0.1
log_peak_memory_stats: False

# Profiler (disabled)
profiler:
  _component_: torchtune.utils.profiler
  enabled: False

Let's run the recipe with this modified config with W&B enabled

In [ ]:
!tune run lora_finetune_single_device --config mistral_wandb_lora.yaml

That's it! Now you can click on the URL and continue monitoring your training on the Weights & Biases UI